In [50]:
import pandas as pd
import numpy as np
import pickle 

# This file contains the amino sequence for each hla allele
oh = pd.read_excel("./For_Ziv_OH.xlsx", index_col=0)

# These files contain beta values for for sequences
coef = pd.read_excel("./For_Ziv_Coefficients.xlsx", index_col=0)
combined = pd.read_excel("./For_Ziv_Combined_Score.xlsx", index_col=0)

# Validate that we can apply matrix multiplication
list(coef.index) == list(oh.columns)

True

In [51]:
# Combines the coefs of the kirs and the coefs of the bindings.
new_coefs = pd.concat([coef, combined], axis=1)
new_coefs.fillna(0, inplace=True)

matrix_oh = oh.values               # one hot of the alleles
matrix_new_coefs = new_coefs.values # the coefs per measure and spot. 

# Multiplies the matrixes to get the result per allele and measure
result = np.matmul(matrix_oh, matrix_new_coefs)

In [52]:
def change_allele_name(allele):
    """
    Gets allele name and adds it a '*' as second character.
    """
    return allele[0] + "*" + allele[1:]
    # return ''.join(list(allele).insert(1, "*"))

def change_measure_name(measure):
    """
    Gets a measure name from the original table, and converts it to Martin's shape.
    """
    return measure.replace("'", "").replace("-", "*")

In [53]:
# This dictionary convert an allele name to a row index in the matrix. 
alleles_places = {change_allele_name(allele.replace("'", "")): i for i, allele in enumerate(oh.index)}

# This dictionary convert a measure name to a column index in the matrix. 
measures_places = {change_measure_name(measure): i for i, measure in enumerate(new_coefs.columns)}

# Now the all the scores of all the alleles are in a matrix, 
# lets put them in a dictionary:
vals = {}

for allele, a_index in alleles_places.items():
    vals[allele] = {}
    for measure, m_index in measures_places.items():
        vals[allele][measure] = result[a_index, m_index]

This section creates functions which handle any amino sequence,
it is not relevant for the goal of this notebook. 

In [54]:
coef_indexes = [c.replace("'", "") for c in coef.index]
coef_indexes = [(int(c[1:]) - 1, c[0]) for c in coef_indexes]

def calculate_seq_vector(seq, coef_indexes):
    seq_vec = np.ndarray((len(coef_indexes),), dtype=np.int64)
    for i, (place, letter) in enumerate(coef_indexes):
        seq_vec[i] = 1 if seq[place] == letter else 0
    
    return seq_vec

def get_seq_result_dict(seq, coef_indexes, matrix_new_coefs):
    seq_oh = calculate_seq_vector(seq, coef_indexes)
    seq_result = np.matmul(seq_oh, matrix_new_coefs)
    seq_result_dict = {}
    for measure, m_index in measures_places.items():
        seq_result_dict[measure] = seq_result[m_index]
    return seq_result_dict

seq = "GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQKMEPRAPWIEQEGPEYWDQETRNMKAHSQTDRANLGTLRGYYNQSEDGSHTIQIMYGCDVGPDGRFLRGYRQDAYDGKDYIALNEDLRSWTAADMAAQITKRKWEAVHAAEQRRVYLEGRCVDGLRRYLENGKETLQRTD"
x = get_seq_result_dict(seq, coef_indexes, matrix_new_coefs)
x

{'KIR3DL1*002': -1.1544792343345809,
 'KIR3DL1*008': -1.1492936500678095,
 'KIR3DL1*029': -0.7325954149749108,
 'KIR3DL1*001': -1.3555063466208699,
 'KIR3DL1*005': -3.1680331892558464,
 'KIR3DL1*015': -0.6262475887547357,
 'KIR3DL1*004': -2.023741544608816,
 'KIR3DL1*009': -1.4950127688177273,
 'KIR3DL1*020': -1.3471345455711288,
 'Score for binding': -5.936199895238416,
 'Score for high low binder': -1.3566148078880782}

Now we have the results, but we would like to know the percentile for each result, to get a context about it:

In [55]:
# We would like to calculate the place (in percents) of the score per allele:

percentiles = {} # The percentiles
sort_values_dict = {} # The values per measure, after sorting 

# The alleles are in the same order like in the results matrix (same order as the excel)
alleles_list = [allele for allele, _ in sorted(alleles_places.items(), key=lambda x: x[1])] # sort them by the places
num = len(alleles_list)

for measure, m_index in measures_places.items():
    # Calculates the percentile of all alleles for measure:
    percentiles[measure] = {}
    
    sort_values = sorted(zip(result[:, m_index], alleles_list))
    sort_values_dict[measure] = [x[0] for x in sort_values] # Keep the sorted values for later
    
    for a_index, (value, allele) in enumerate(sort_values):
        percentiles[measure][allele] = a_index / num

In [56]:
# But we want the dictionary to be in the same structure as the former one (dict[allele][measure])
temp = {}
for allele in alleles_places:
    temp[allele] = {}
    for measure in measures_places:
        temp[allele][measure] = percentiles[measure][allele]

percentiles = temp

# Threshold
Now we would like to calculates threshold values, 
they are defined ad the avergae between A*23:01 scores and B*27:05 scores. 

In [57]:
# Set threshold as values (and not percents)

thresholds_vals = {}
thresholds_percentile = {}

a = "A*23:01"
b = "B*27:05"

for measure in measures_places:
    thresholds_vals[measure] = (vals[a][measure] + vals[b][measure]) / 2

thresholds_vals

{'KIR3DL1*002': 0.5815396583350322,
 'KIR3DL1*008': 0.567616225098019,
 'KIR3DL1*029': 0.5510481934498993,
 'KIR3DL1*001': 0.47395733946011004,
 'KIR3DL1*005': 0.03182070601928455,
 'KIR3DL1*015': 0.4608754218574506,
 'KIR3DL1*004': 0.150471532207204,
 'KIR3DL1*009': 0.4352646721612451,
 'KIR3DL1*020': 0.5029622128602422,
 'Score for binding': 6.062997153966869,
 'Score for high low binder': 3.816529272154212}

In [58]:
def get_percentile(sort_values_dict, scores):
    percentiles_for_this = {}
    
    for measure in scores:
        arr = sort_values_dict[measure]
        t_val = scores[measure]
        num = len(arr) - 1 

        for index, val in enumerate(arr):
            if t_val < val:
                percentiles_for_this[measure] = index / num
                break

        # If this is the biggest val in arr, so it is in percentile 1.
        if measure not in percentiles_for_this:
            percentiles_for_this[measure] = 1    
    
    return percentiles_for_this

# Gets the percentiles of the threshold values. 
thresholds_percentile = get_percentile(sort_values_dict, thresholds_vals)
thresholds_percentile

{'KIR3DL1*002': 0.8323406412368521,
 'KIR3DL1*008': 0.8320871879356229,
 'KIR3DL1*029': 0.8308199214294766,
 'KIR3DL1*001': 0.7266506146242555,
 'KIR3DL1*005': 0.3139019135724243,
 'KIR3DL1*015': 0.8315802813331644,
 'KIR3DL1*004': 0.5441642377391965,
 'KIR3DL1*009': 0.6815359270054493,
 'KIR3DL1*020': 0.7388163730832594,
 'Score for binding': 0.8310733747307059,
 'Score for high low binder': 0.8081358509694588}

Write all the results into pickle files, so the website will be able to use them:

In [59]:
import os

os.makedirs("../pkl", exist_ok=True)

with open("../pkl/vals.pkl", "wb") as f:
    pickle.dump(vals, f)

with open("../pkl/percentiles.pkl", "wb") as f:
    pickle.dump(percentiles, f)

with open("../pkl/threshold_vals.pkl", "wb") as f:
    pickle.dump(thresholds_vals, f)

with open("../pkl/threshold_percentile.pkl", "wb") as f:
    pickle.dump(thresholds_percentile, f)

with open("../pkl/sort_values.pkl", "wb") as f:
    pickle.dump(sort_values_dict, f)

with open("../pkl/coefs_indexes.pkl", "wb") as f:
    pickle.dump(coef_indexes, f)

with open("../pkl/matrix_new_coefs.pkl", "wb") as f:
    pickle.dump(matrix_new_coefs, f)

with open("../pkl/measures_places.pkl", "wb") as f:
    pickle.dump(measures_places, f)